# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298315226230                   -0.85    5.0         
  2   -8.300179724355       -2.73       -1.25    1.1   84.5ms
  3   -8.300428852429       -3.60       -1.89    3.0    110ms
  4   -8.300461268081       -4.49       -2.75    2.0    214ms
  5   -8.300463502183       -5.65       -3.00    4.4    122ms
  6   -8.300464092620       -6.23       -3.21    5.4    135ms
  7   -8.300464374373       -6.55       -3.36    2.5    109ms
  8   -8.300464523182       -6.83       -3.51    1.2   90.5ms
  9   -8.300464615579       -7.03       -3.74    1.6    107ms
 10   -8.300464623206       -8.12       -3.82    1.0   92.4ms
 11   -8.300464640553       -7.76       -4.08    2.5    133ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67400668255                   -0.70    5.9         
  2   -16.67851384569       -2.35       -1.14    1.2    187ms
  3   -16.67918195568       -3.18       -1.86    3.4    308ms
  4   -16.67925807897       -4.12       -2.70    3.2    237ms
  5   -16.67928524848       -4.57       -3.15    4.8    303ms
  6   -16.67928618483       -6.03       -3.45    2.4    246ms
  7   -16.67928621373       -7.54       -3.90    1.6    188ms
  8   -16.67928622121       -8.13       -4.54    3.1    246ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32626015412                   -0.56    7.1         
  2   -33.33438243508       -2.09       -1.00    1.4    701ms
  3   -33.33590415705       -2.82       -1.72    6.1    945ms
  4   -33.33612473242       -3.66       -2.63    2.5    801ms
  5   -33.33676011821       -3.20       -2.19    8.2    1.18s
  6   -33.33679331511       -4.48       -2.24    1.0    619ms
  7   -33.33694308110       -3.82       -3.39    2.2    761ms
  8   -33.33694369487       -6.21       -3.62    5.5    1.02s
  9   -33.33694362971   +   -7.19       -3.68    2.2    757ms
 10   -33.33694378097       -6.82       -4.58    2.2    744ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298594969697                   -0.85    5.1         
  2   -8.300283737467       -2.77       -1.58    1.0   60.9ms
  3   -8.300435110843       -3.82       -2.62    2.5   78.5ms
  4   -8.300407628204   +   -4.56       -2.37    3.4    112ms
  5   -8.300464392317       -4.25       -3.41    1.1   63.3ms
  6   -8.300464610827       -6.66       -3.94    2.4   89.0ms
  7   -8.300464641290       -7.52       -4.31    1.6   75.0ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32684440536                   -0.56    7.1         
  2   -33.33377358122       -2.16       -1.28    1.4    570ms
  3   -33.09554999332   +   -0.62       -1.28    7.8    1.07s
  4   -30.74229505474   +    0.37       -0.86    4.4    981ms
  5   -33.15853823266        0.38       -1.33    4.0    881ms
  6   -33.33142122936       -0.76       -2.06    2.0    695ms
  7   -33.04656466636   +   -0.55       -1.33    4.2    1.06s
  8   -33.33398139762       -0.54       -2.24    4.8    949ms
  9   -33.33593585685       -2.71       -2.38    1.9    670ms
 10   -33.33656049642       -3.20       -2.62    1.9    622ms
 11   -33.33667582783       -3.94       -2.71    1.9    625ms
 12   -33.33689977810       -3.65      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.